<a href="https://colab.research.google.com/github/shivamsouravjha/-PetFinder.my-Adoption-Prediction-/blob/master/Petfinder_Using_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import os
from keras.layers import Dense,BatchNormalization
from sklearn.model_selection import train_test_split

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
trainX = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')
print(trainX.shape)
print(test.shape)
trainX.head()


In [ ]:
for col in trainX.columns:
    if trainX[col].dtype=='object':
        trainX.drop([col],axis=1,inplace=True)

In [ ]:
for col in test.columns:
    if test[col].dtype=='object':
        test.drop([col],axis=1,inplace=True)

In [ ]:
trainY = pd.DataFrame()
trainY['AdoptionSpeed'] = trainX['AdoptionSpeed']
trainX=trainX.drop(['AdoptionSpeed','State',	'VideoAmt'	,'PhotoAmt'],axis=1)
test=test.drop(['State','VideoAmt'	,'PhotoAmt'],axis=1)

In [ ]:
print(trainX.shape)
trainX.head()


In [ ]:
print(test.shape)
test.head()


In [ ]:
test_image = os.listdir('../input/petfinder-adoption-prediction/test_images')
train_image = os.listdir('../input/petfinder-adoption-prediction/train_images')

In [ ]:
def display_samples(columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 3*rows))
    for i in range(columns*rows):
        image_path = train_image[i]
        img = cv2.imread(f'../input/petfinder-adoption-prediction/train_images/{image_path}')
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(img)

display_samples()


In [ ]:
from keras.models import Sequential

In [ ]:
model  = Sequential()

In [ ]:
trainX.shape

In [ ]:
model.add(Dense(4096,activation= 'relu',input_shape=(16,))) #dense layer 1
model.add(tf.keras.layers.BatchNormalization()) #BachNorm
model.add(tf.keras.layers.Dropout(0.25)) #Dropout
model.add(Dense(2048,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout
model.add(Dense(1024,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout
model.add(Dense(512,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.35)) #Dropout
model.add(Dense(256,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout

model.add(Dense(128,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout

model.add(Dense(64,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.35)) #Dropout
model.add(BatchNormalization())

model.add(Dense(32,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout

model.add(Dense(16,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.15)) #Dropout

model.add(Dense(5,activation= 'softmax'))  

In [ ]:
from keras.optimizers import RMSprop,Adam,SGD,Adagrad,Adadelta,Adamax,Nadam

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.99)
# Compile the model
model.compile(optimizer = optimizer , loss = 'categorical_crossentropy', metrics=["accuracy"])


In [ ]:
trainX =trainX.values

In [ ]:
trainY=trainY.values

In [ ]:
valx,testx,valy,testy = train_test_split(trainX,trainY,test_size=0.25)

In [ ]:
testy = tf.keras.utils.to_categorical(testy, 5)
valy = tf.keras.utils.to_categorical(valy, 5)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.37):
            print("\nReached 40.0% accuracy so cancelling training!")
            self.model.stop_training = True


In [ ]:
callback = myCallback()

In [ ]:
model.fit(testx,testy,validation_data =(valx,valy),batch_size=750,epochs=256,callbacks = [callback])

In [ ]:
scores = model.evaluate(valx, valy, verbose=0)
print("{}: {:.2f}%".format("accuracy", scores[1]*100))

In [ ]:
model.summary()

In [ ]:
print(test.shape)

In [ ]:
test.head()

In [ ]:
prc =model.predict(test)


In [ ]:
print(prc)

In [ ]:
prc = prc.argmax(axis=1)

In [ ]:
print(prc)

In [ ]:
sample = pd.read_csv('../input/petfinder-adoption-prediction/test/sample_submission.csv')

In [ ]:
sample.AdoptionSpeed = prc

In [ ]:
sample.head(50)

In [ ]:
sample.to_csv('submission.csv', index=False)